# Mediator Recipe

### imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
import asyncio
from termcolor import colored

## Initialise Agent

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://mediator-agent:3021 and an api key of adminApiKey


In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Listeners

In [4]:

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "checkRecords":
        # Your business logic
        print("check")
    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)




### Create Multi-Use Mediation Invitation

Note you could create single use invitations too, but likely as a mediator you would want the invitation to be multi-use

In [5]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(multi_use="true")
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection 85a76bd1-b5dd-48d5-b5a8-feab72d0f4a4 in State invitation
invitation
Connection ID 85a76bd1-b5dd-48d5-b5a8-feab72d0f4a4
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '0ea5e233-7d65-41b6-bf39-d22c3b6bc6f8', 'label': 'MediatorAgent', 'serviceEndpoint': 'https://8dee682d2e61.ngrok.io', 'recipientKeys': ['3cVWmAw4Y6gLt2e2B3qEHuXHKx3rQh761F4XZ5tFgXsZ']}


## Check Mediation Requests

In [ ]:
### To have a look at the mediation records and we should see our mediation in there
records_response = await agent_controller.mediation.get_mediation_records()
print(records_response)


## Grant Mediation Requests

If not handled automatically

In [ ]:
grant = await agent_controller.mediation.grant_mediation_request_by_id(records_response[0]["mediation_id"])
print(grant)

## Terminate Controller

In [6]:
await agent_controller.terminate()